In [10]:
!echo $PYTHONPATH

/Users/simon.hughes/anaconda3/envs/phd_py36/bin:/Users/simon.hughes/anaconda3/envs/phd_py36/bin:/Users/simon.hughes/anaconda3/envs/work/bin:/Users/simon.hughes/anaconda3/bin:/opt/local/bin:/opt/local/sbin:/Users/simon.hughes/.pyenv/bin:/Users/simon.hughes/ShellScripts:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/usr/local/MacGPG2/bin


In [11]:
!export export PYTHONPATH='/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/PythonNlpResearch:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Calculations:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Classifiers:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Classifiers/RegEx:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Classifiers/Trees:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Clustering:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/CodeGen:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/CollectionsHelper:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/Chicago:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/CoralBleaching:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/CoralBleachingWordTagger:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/SkinCancerWordTagger:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/GlobalWarming:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/GlobalWarmingAnnotated:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Data/GoogleNGrams:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/DeepBeliefNetwork:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/Examples:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/Examples/DeepAutoEncoder:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/Examples/DeepAutoEncoder_MichaelNielsen:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/MyCode:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/MyCode/RecursiveAutoEncoder:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/WordVectors:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/Chicago:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/CoralBleaching:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/GlobalWarming:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/GlobalWarming/Causal:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/GlobalWarming/WordClustering:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/FeatureExtraction/Text:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Frequency:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/LanguageModel:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Results:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Tagging:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/VectorSpace:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/VectorSpace/WordVectors:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/VectorSpace/WordVectors/Word2Vec:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/_Scratch/Sub:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Synonyms:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/DeepLearning/MyCode/:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/MontyLingua:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/CoralBleachingWordTagger/WindowBasedClassifier:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/Tagging:/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/Experiments/CoralBleachingWordTagger/DeepNN'

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics_causal

In [3]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [4]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(lambda s: str(s),vals)).encode('utf-8') 
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
    #        "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
    #        "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision" ] + params
        return df[cols]
    return df

In [13]:
def __precision__(tp, fp, fn):
    if tp + fp <= 0:
        return 0.0
    return tp / (tp + fp)

def __recall__(tp, fp, fn):
    if tp + fn <= 0:
        return 0.0
    return tp / (tp + fn)

def __f_beta__(r, p, beta):
    if r + p <= 0.0:
        return 0.0
    beta_squared = beta * beta
    #Harmonic mean
    return ((1.0 + beta_squared) * r * p) / (beta_squared * (r + p))

def rpf1a_from_tp_fp_tn_fn(tp, fp, tn, fn):
    r = __recall__(tp, fp, fn)
    p = __precision__(tp, fp, fn)
    f1 = __f_beta__(r, p, 1.0)

    a = (tp + tn) / (tp + fp + tn + fn)
    return (r, p, f1, a)


In [14]:
#from Metrics import rpf1a_from_tp_fp_tn_fn
from collections import defaultdict

def tally_counts(r, filter):
    tally = defaultdict(int)
    for k,v in r.items():
        if filter(k):
            for prop in "tp,tn,fp,fn".split(","):
                tally[prop] += v[prop]
    return tally

def get_causal_relation_metrics(collection, params, include_concept_codes=True):
    dicts = []
    for r in db[collection].find({}):
        d = {}
        cr_counts = tally_counts(r, lambda c: "->" in c)
        (rec, p, cr_f1, a) = rpf1a_from_tp_fp_tn_fn(cr_counts["tp"],cr_counts["fp"],cr_counts["tn"],cr_counts["fn"])
        d["cr_micro_f1"] = cr_f1
        d["cr_micro_rec"]  = rec
        d["cr_micro_prec"] = p
        if include_concept_codes:
            concept_counts = tally_counts(r, lambda c: c[0].isdigit())
            (rec, p, concept_f1, a) = rpf1a_from_tp_fp_tn_fn(concept_counts["tp"],concept_counts["fp"],concept_counts["tn"],concept_counts["fn"])
            d["concept_micro_f1"] = concept_f1
            d["concept_micro_rec"]  = rec
            d["concept_micro_prec"] = p
        parms = r["parameters"]
        for p in params:
            d[p] = parms[p]
        dicts.append(d)
    df = pd.DataFrame(dicts)
    fields = ("cr_micro_f1,cr_micro_rec,cr_micro_prec,concept_micro_f1,concept_micro_rec,concept_micro_prec," + ",".join(params)).split(",")
    if not include_concept_codes:
        fields = [f for f in fields if "concept" not in f]
    return df[fields].sort_values("cr_micro_f1", ascending=False)

In [15]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

# Binary Relevance - CR and Concept Codes

## Coral Bleaching
(no skin cancer results for this experiment)

### <span style="color:red">No longer valid - not included in final experiments</span>

# Most Common Tag

## Coral Bleaching

In [16]:
params = "merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size"

### Training

In [17]:
df = get_df_sorted_by_f1score("CR_CB_TAGGING_TD_MOST_COMMON_TAG_RNN", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.8688,0.8714,0.8662,sum,2,True,True,256
1,0.8054,0.8128,0.7980,sum,2,True,True,128
2,0.7852,0.7980,0.7729,sum,1,True,True,256
3,0.7542,0.8007,0.7128,sum,1,True,True,128
4,0.7418,0.7381,0.7455,sum,2,True,True,64
5,0.7169,0.7509,0.6858,sum,1,True,True,64


### Validation

In [18]:
df = get_df_sorted_by_f1score("CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.6798,0.6949,0.6653,sum,2,True,True,256
1,0.6731,0.6803,0.6661,sum,2,True,True,128
2,0.6498,0.6564,0.6433,sum,1,True,True,256
3,0.6358,0.6790,0.5978,sum,1,True,True,128
4,0.6351,0.6540,0.6173,sum,1,True,True,64
5,0.6342,0.6281,0.6404,sum,2,True,True,64


### Test

In [19]:
df = get_df_sorted_by_f1score("TEST_CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.6764,0.6561,0.6980,sum,2,True,True,256


## Skin Cancer

### Training

In [20]:
df = get_df_sorted_by_f1score("CR_SC_TAGGING_TD_MOST_COMMON_TAG_RNN", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.8718,0.8549,0.8894,sum,2,True,True,256
1,0.8527,0.8534,0.8520,sum,2,True,True,128
2,0.8494,0.8380,0.8610,sum,2,True,True,64
3,0.8423,0.8369,0.8478,sum,1,True,True,256
4,0.8325,0.8569,0.8095,sum,1,True,True,128
5,0.7996,0.8188,0.7812,sum,1,True,True,64


### Validation

In [21]:
df = get_df_sorted_by_f1score("CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.7686,0.7608,0.7765,sum,2,True,True,256
1,0.7562,0.7628,0.7497,sum,2,True,True,128
2,0.7539,0.7545,0.7533,sum,1,True,True,256
3,0.7521,0.7415,0.7631,sum,2,True,True,64
4,0.7418,0.7726,0.7134,sum,1,True,True,128
5,0.7295,0.7499,0.7102,sum,1,True,True,64


### Test

In [22]:
df = get_df_sorted_by_f1score("TEST_CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN", params)
round_data(df,4)

,micro_f1_score,micro_recall,micro_precision,merge_mode,num_rnns,use_pretrained_embedding,bi-directional,hidden_size
0,0.7918,0.7979,0.7859,sum,2,True,True,256


## Stacked Model

### Coral Bleaching

### Training

In [23]:
sparams = "dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats".split(",")
df = get_df_sorted_by_f1score("CR_CB_STACKED_TD", sparams)
round_data(df,4).head(5)

,micro_f1_score,micro_recall,micro_precision,dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats
0,0.7415,0.7113,0.7742,False,l1,100.0,False,False,False,True,True
1,0.7412,0.7106,0.7745,False,l2,100.0,False,False,False,True,True
2,0.7407,0.7096,0.7747,False,l1,10.0,False,False,False,True,True
3,0.7387,0.7072,0.7733,False,l2,10.0,False,False,False,True,True
4,0.7386,0.7068,0.7733,True,l2,10.0,False,False,False,True,True


### Validation

In [24]:
df = get_df_sorted_by_f1score("CR_CB_STACKED_VD", sparams)
round_data(df,4).head(5)

,micro_f1_score,micro_recall,micro_precision,dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats
0,0.6946,0.6560,0.7380,False,l1,1.0,False,False,False,True,True
3,0.6936,0.6490,0.7447,False,l2,1.0,False,False,False,True,True
1,0.6936,0.6490,0.7447,True,l2,1.0,False,False,False,True,True
2,0.6936,0.6490,0.7447,True,l2,1.0,False,False,False,True,True
4,0.6930,0.6567,0.7336,False,l1,0.5,False,False,False,True,True


### Test

In [25]:
df = get_df_sorted_by_f1score("TEST_CR_CB_STACKED_VD", sparams)
round_data(df,4).head(5)

,micro_f1_score,micro_recall,micro_precision,dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats
0,0.7038,0.6745,0.7359,False,l1,1.0,False,False,False,True,True


### Skin Cancer

### Training

In [26]:
sparams = "dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats".split(",")
df = get_df_sorted_by_f1score("CR_SC_STACKED_TD", sparams)
round_data(df,4).head(5)

,micro_f1_score,micro_recall,micro_precision,dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats
0,0.8007,0.7566,0.8504,False,l1,100.0,True,False,True,False,True
1,0.7952,0.7492,0.8473,False,l2,100.0,True,False,True,False,True
2,0.7929,0.7459,0.8464,False,l1,10.0,True,False,True,False,True
3,0.7891,0.7407,0.8444,False,l1,5.0,True,False,True,False,True
4,0.7888,0.7407,0.8436,False,l2,10.0,True,False,True,False,True


### Validation

In [27]:
sparams = "dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats".split(",")
df = get_df_sorted_by_f1score("CR_SC_STACKED_VD", sparams)
round_data(df,10).head(5)

,micro_f1_score,micro_recall,micro_precision,dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats
0,0.7631578947,0.7112987535,0.8231735691,True,l2,1.0,True,False,True,False,True
1,0.7631578947,0.7112987535,0.8231735691,True,l2,1.0,False,True,True,False,True
2,0.7631578947,0.7112987535,0.8231735691,True,l2,1.0,True,False,True,False,True
3,0.7631578947,0.7112987535,0.8231735691,False,l2,1.0,True,False,True,False,True
4,0.7629509962,0.7121029353,0.8216191139,True,l2,1.0,True,False,True,True,True


### Test

In [28]:
sparams = "dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats".split(",")
df = get_df_sorted_by_f1score("TEST_CR_SC_STACKED_TD", sparams)
round_data(df,4).head(5)

,micro_f1_score,micro_recall,micro_precision,dual,penalty,C,max_feats,min_feats,average_feats,binary_feats,combo_feats
0,0.7762,0.7236,0.8372,True,l2,1.0,True,False,False,True,True


## S-R Parser

In [29]:
def extract_parameter(s, param_name):
    s = s.replace("("," ").replace(")"," ")
    keys = s.split(" ")
    return [(key,val.replace(",","").replace("'","")) for key,val in [k.split("=") for k in keys if "=" in k] if key == param_name][0][-1]

extract_c_val = lambda s: extract_parameter(s, "C")
extract_penalty_val = lambda s: extract_parameter(s, "penalty")
extract_dual_val = lambda s: extract_parameter(s, "dual")

s = "LogisticRegression(C=0.1, class_weight=None, dual=True, fit_intercept=True, intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1, penalty='l2', random_state=None, solver='liblinear', tol=0.0001, verbose=0, warm_start=False)"
extract_c_val(s)

'0.1'

### Coral Bleaching

### Training

In [30]:
col = ["CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_TD"]

df_feat_sel = get_df_sorted_by_f1score(col[0], "algorithm,beta,max_epochs", filter_cols=True) 
df_feat_sel["C"] = df_feat_sel["algorithm"].apply(extract_c_val)
df_feat_sel["penalty"] = df_feat_sel["algorithm"].apply(extract_penalty_val)
df_feat_sel["dual"] = df_feat_sel["algorithm"].apply(extract_dual_val)
df_feat_sel.head(10)

,micro_f1_score,micro_recall,micro_precision,algorithm,beta,max_epochs,C,penalty,dual
0,0.864852,0.766384,0.992354,"LogisticRegression(C=10.0, class_weight=None, ...",0.1,20,10.0,l2,False
1,0.864812,0.766384,0.992247,"LogisticRegression(C=10.0, class_weight=None, ...",0.3,15,10.0,l2,False
2,0.864758,0.766301,0.992246,"LogisticRegression(C=10.0, class_weight=None, ...",0.5,15,10.0,l2,False
3,0.864733,0.766134,0.992459,"LogisticRegression(C=10.0, class_weight=None, ...",0.3,20,10.0,l2,False
4,0.864692,0.766134,0.992352,"LogisticRegression(C=10.0, class_weight=None, ...",0.4,15,10.0,l2,False
5,0.864598,0.766051,0.992244,"LogisticRegression(C=10.0, class_weight=None, ...",0.4,20,10.0,l2,False
6,0.864586,0.765968,0.992350,"LogisticRegression(C=10.0, class_weight=None, ...",0.5,20,10.0,l2,False
7,0.864571,0.766134,0.992031,"LogisticRegression(C=10.0, class_weight=None, ...",0.4,10,10.0,l2,False
8,0.864474,0.766301,0.991499,"LogisticRegression(C=10.0, class_weight=None, ...",0.4,20,10.0,l2,True
9,0.864457,0.766717,0.990758,"LogisticRegression(C=10.0, class_weight=None, ...",0.2,20,10.0,l2,True


### Validation

In [32]:
col = ["CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD"]

df_feat_sel = get_df_sorted_by_f1score(col[0], "algorithm,beta,max_epochs", filter_cols=True) 
df_feat_sel["C"] = df_feat_sel["algorithm"].apply(extract_c_val)
df_feat_sel["penalty"] = df_feat_sel["algorithm"].apply(extract_penalty_val)
df_feat_sel["dual"] = df_feat_sel["algorithm"].apply(extract_dual_val)
df_feat_sel.head(10)

,micro_f1_score,micro_recall,micro_precision,algorithm,beta,max_epochs,C,penalty,dual
0,0.720357,0.670991,0.777564,"LogisticRegression(C=0.1, class_weight=None, d...",0.5,5,0.1,l2,True
1,0.720357,0.670991,0.777564,"LogisticRegression(C=0.1, class_weight=None, d...",0.5,5,0.1,l2,False
2,0.720242,0.672322,0.775518,"LogisticRegression(C=0.1, class_weight=None, d...",0.5,10,0.1,l2,True
3,0.720242,0.672322,0.775518,"LogisticRegression(C=0.1, class_weight=None, d...",0.5,10,0.1,l2,False
4,0.720100,0.667997,0.781019,"LogisticRegression(C=0.5, class_weight=None, d...",0.2,20,0.5,l2,True
5,0.720100,0.667997,0.781019,"LogisticRegression(C=0.5, class_weight=None, d...",0.2,20,0.5,l2,False
6,0.719943,0.671324,0.776154,"LogisticRegression(C=0.1, class_weight=None, d...",0.4,5,0.1,l2,True
7,0.719943,0.671324,0.776154,"LogisticRegression(C=0.1, class_weight=None, d...",0.4,5,0.1,l2,False
9,0.719715,0.670991,0.776068,"LogisticRegression(C=0.1, class_weight=None, d...",0.3,15,0.1,l2,False
8,0.719715,0.670991,0.776068,"LogisticRegression(C=0.1, class_weight=None, d...",0.3,15,0.1,l2,True


### Test

In [33]:
col = ["TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD"]

df_feat_sel = get_df_sorted_by_f1score(col[0], "algorithm,beta,max_epochs", filter_cols=True) 
df_feat_sel["C"] = df_feat_sel["algorithm"].apply(extract_c_val)
df_feat_sel["penalty"] = df_feat_sel["algorithm"].apply(extract_penalty_val)
df_feat_sel["dual"] = df_feat_sel["algorithm"].apply(extract_dual_val)
df_feat_sel.head(10)

,micro_f1_score,micro_recall,micro_precision,algorithm,beta,max_epochs,C,penalty,dual
0,0.727744,0.702838,0.75448,"LogisticRegression(C=0.1, class_weight=None, d...",0.5,5,0.1,l2,True


### Skin Cancer

### Training

In [34]:
col = ["CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_TD"]

df_feat_sel = get_df_sorted_by_f1score(col[0], "algorithm,beta,max_epochs", filter_cols=True) 
df_feat_sel["C"] = df_feat_sel["algorithm"].apply(extract_c_val)
df_feat_sel["penalty"] = df_feat_sel["algorithm"].apply(extract_penalty_val)
df_feat_sel["dual"] = df_feat_sel["algorithm"].apply(extract_dual_val)
df_feat_sel.head(10)

,micro_f1_score,micro_recall,micro_precision,algorithm,beta,max_epochs,C,penalty,dual
0,0.879436,0.789706,0.992170,"LogisticRegression(C=100.0, class_weight=None,...",0.2,20,100.0,l2,False
1,0.879350,0.789254,0.992667,"LogisticRegression(C=100.0, class_weight=None,...",0.5,20,100.0,l2,False
2,0.879277,0.789807,0.991607,"LogisticRegression(C=100.0, class_weight=None,...",0.3,20,100.0,l2,False
3,0.879254,0.789455,0.992105,"LogisticRegression(C=100.0, class_weight=None,...",0.3,10,100.0,l2,False
4,0.879230,0.789455,0.992042,"LogisticRegression(C=100.0, class_weight=None,...",0.5,15,100.0,l2,False
5,0.879209,0.789304,0.992228,"LogisticRegression(C=100.0, class_weight=None,...",0.4,20,100.0,l2,False
6,0.879196,0.789204,0.992353,"LogisticRegression(C=100.0, class_weight=None,...",0.4,15,100.0,l2,False
7,0.879163,0.789505,0.991792,"LogisticRegression(C=100.0, class_weight=None,...",0.1,20,100.0,l2,False
8,0.879111,0.789304,0.991978,"LogisticRegression(C=100.0, class_weight=None,...",0.2,10,100.0,l2,False
9,0.879089,0.789505,0.991604,"LogisticRegression(C=100.0, class_weight=None,...",0.5,10,100.0,l2,False


### Validation

In [35]:
col = ["CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD"]

df_feat_sel = get_df_sorted_by_f1score(col[0], "algorithm,beta,max_epochs", filter_cols=True) 
df_feat_sel["C"] = df_feat_sel["algorithm"].apply(extract_c_val)
df_feat_sel["penalty"] = df_feat_sel["algorithm"].apply(extract_penalty_val)
df_feat_sel["dual"] = df_feat_sel["algorithm"].apply(extract_dual_val)
df_feat_sel.head(10)

,micro_f1_score,micro_recall,micro_precision,algorithm,beta,max_epochs,C,penalty,dual
0,0.766909,0.710092,0.833609,"LogisticRegression(C=0.5, class_weight=None, d...",0.2,5,0.5,l2,False
1,0.766800,0.712304,0.830326,"LogisticRegression(C=0.5, class_weight=None, d...",0.4,20,0.5,l2,False
2,0.766602,0.712505,0.829588,"LogisticRegression(C=0.5, class_weight=None, d...",0.5,10,0.5,l2,False
3,0.766580,0.711098,0.831453,"LogisticRegression(C=0.5, class_weight=None, d...",0.3,20,0.5,l2,False
4,0.766512,0.715119,0.825865,"LogisticRegression(C=0.5, class_weight=None, d...",0.1,20,0.5,l1,False
5,0.766396,0.710696,0.831569,"LogisticRegression(C=0.5, class_weight=None, d...",0.2,15,0.5,l2,True
6,0.766367,0.711902,0.829857,"LogisticRegression(C=0.5, class_weight=None, d...",0.4,20,0.5,l2,True
7,0.766363,0.710897,0.831218,"LogisticRegression(C=0.5, class_weight=None, d...",0.4,10,0.5,l2,True
8,0.766311,0.714314,0.826471,"LogisticRegression(C=0.1, class_weight=None, d...",0.2,5,0.1,l2,True
9,0.766266,0.711500,0.830167,"LogisticRegression(C=0.5, class_weight=None, d...",0.3,15,0.5,l2,True


# Compare Top Validation Metrics By Algorithm

### Coral Bleaching

In [36]:
cb_collections = [
    "CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN", 
    "CR_CB_STACKED_VD",
    "CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD"
]

In [37]:
rows = []
for coll in cb_collections:
    df = get_df_sorted_by_f1score(coll, "")
    dct = df.iloc[0].to_dict()
    dct["Algo"] = coll
    rows.append(dct)

df=pd.DataFrame(rows)
df.sort_values("micro_f1_score", ascending=False)

,Algo,micro_f1_score,micro_precision,micro_recall
2,CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARA...,0.720357,0.777564,0.670991
1,CR_CB_STACKED_VD,0.694611,0.738024,0.656021
0,CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN,0.679792,0.665287,0.694943


### Skin Cancer

In [38]:
rows = []
for coll in cb_collections: 
    coll = coll.replace("CB", "SC")
    df = get_df_sorted_by_f1score(coll, "")
    dct = df.iloc[0].to_dict()
    dct["Algo"] = coll
    rows.append(dct)

df=pd.DataFrame(rows)
df.sort_values("micro_f1_score", ascending=False)

,Algo,micro_f1_score,micro_precision,micro_recall
0,CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN,0.768559,0.776524,0.760756
2,CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARA...,0.766909,0.833609,0.710092
1,CR_SC_STACKED_VD,0.763158,0.823174,0.711299


## Test Data

In [39]:
# Parser model not present yet
test_collections = ["TEST_" + c.replace("HYPER_PARAM_","") for c in cb_collections]
cb_collections

['CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN',
 'CR_CB_STACKED_VD',
 'CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD']

In [40]:
# Still need this for RNN and the parser model
rows = []
for coll in test_collections:

    df = get_df_sorted_by_f1score(coll, "")
    dct = df.iloc[0].to_dict()
    dct["Algo"] = coll
    rows.append(dct)

df=pd.DataFrame(rows)
df.sort_values("micro_f1_score", ascending=False)

,Algo,micro_f1_score,micro_precision,micro_recall
2,TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD,0.727744,0.754480,0.702838
1,TEST_CR_CB_STACKED_VD,0.703833,0.735883,0.674457
0,TEST_CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN,0.676420,0.698046,0.656093


In [41]:
# Still need this for RNN and the parser model
rows = []
for coll in test_collections:
    
    coll = coll.replace("CB", "SC")
    df = get_df_sorted_by_f1score(coll, "")
    dct = df.iloc[0].to_dict()
    dct["Algo"] = coll
    rows.append(dct)

df=pd.DataFrame(rows)
df.sort_values("micro_f1_score", ascending=False)

,Algo,micro_f1_score,micro_precision,micro_recall
0,TEST_CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN,0.791833,0.785903,0.797853
2,TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_VD,0.790330,0.822846,0.760286
1,TEST_CR_SC_STACKED_VD,0.764622,0.816244,0.719141


# Print LaTex Tables by Algorithm

In [42]:
def print_row(row, algo):
    macro_f1 = row["MICRO_F1"]["f1_score"]
    mprec = row["MICRO_F1"]["precision"]
    mrec =  row["MICRO_F1"]["recall"]
    print("{algo} &\t{mf1:.3f}\t\t\t&\t{rec:.3f}\t\t\t&\t{prec:.3f} \\\\".format(algo=algo.ljust(25), mf1=macro_f1, prec=mprec, rec=mrec))    

def print_micro_table_row(coll, algo):
    for row in db[coll].find({}):
        print_row(row, algo)

In [43]:
# Where multiple rows (e.g. in CV hyper parameter tuning) grab the best only
def print_best_micro_table_row(coll, algo):
    best_f1 = -1
    best_tuple = None
    for row in db[coll].find({}):        
        micro_f1 = row["MICRO_F1"]["f1_score"]
        if micro_f1 > best_f1:
            best_f1 = micro_f1
            best_tuple = (micro_f1, row["MICRO_F1"]["precision"], row["MICRO_F1"]["recall"])
    
    micro_f1, mprec, mrec = best_tuple
    print("{algo} &\t{mf1:.3f}\t\t\t&\t{rec:.3f}\t\t\t&\t{prec:.3f} \\\\".format(algo=algo.ljust(25), mf1=micro_f1, prec=mprec, rec=mrec))    

In [44]:
def get_variants(colname):
    return [
        colname.replace("VD","TD"), 
        colname.replace("TD","VD"), 
        "TEST_" + colname.replace("TD","VD")
    ]

def get_algo_name(coll):
    if "RNN" in coll:
        return "Bidirectional RNN"
    if "STACKED" in coll:
        return "Stacked Model"
    if "SHIFT_REDUCE" in coll:
        return "Shift Reduce Parser"
    raise Exception("Unknow algo name for collection: {col}".format(col=coll))
    
def get_data_partition_name(coll):
    if "TEST_" in coll:
        if "VD" in coll:
            return "Test Data"
        else:
            raise Exception("Unknow data partition name for collection: {col}".format(col=coll))
    if "_TD" in coll:
        return "Training Data (CV)"
    if "_VD" in coll:
        return "Validation Data (CV)"
    raise Exception("Unknow data partition name for collection: {col}".format(col=coll))
    
def get_dataset_name(coll):
    if "SC" in coll:
        return "Skin Cancer"
    else:
        return "Coral Bleaching"

In [45]:
get_variants(cb_collections[0])

['CR_CB_TAGGING_TD_MOST_COMMON_TAG_RNN',
 'CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN',
 'TEST_CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN']

In [46]:
def print_table_header( caption):
    print("""
\\begin{table}[H]
\\caption{""" + caption + """}
\\center{
\\begin{tabular}{l r r r}
\\toprule
\\textbf{Data Partition} & \\textbf{Micro-$F_1$} & \\textbf{Micro-Recall} & \\textbf{Micro-Precision}\\\\
\\midrule
""")

In [47]:
print_table_header("CAPTION")


\begin{table}[H]
\caption{CAPTION}
\center{
\begin{tabular}{l r r r}
\toprule
\textbf{Data Partition} & \textbf{Micro-$F_1$} & \textbf{Micro-Recall} & \textbf{Micro-Precision}\\
\midrule



In [48]:
def print_table_footer(label):
    print("""
\\bottomrule
\\end{tabular}}
\\label{table:""" + label + """}
\\end{table}

""")
    

In [49]:
print_table_footer("label")


\bottomrule
\end{tabular}}
\label{table:label}
\end{table}




In [50]:
def get_crel_caption(coll):
    return get_algo_name(coll) + " Causal Relation Classification Accuracy on the " + get_dataset_name(coll) + " Dataset"

def get_crel_lbl(coll):
    suffix = "_cb" if "CB" in coll else "_sc"
    return "rq2_" + get_algo_name(coll).replace(" ","_").lower() + suffix

In [51]:
def print_table(colls):
    caption= get_crel_caption(colls[0])
    print_table_header(caption)
    for col in colls:
        #print(col)
        print_best_micro_table_row(col, get_data_partition_name(col))
        #print(col)
    print_table_footer(label=get_crel_lbl(colls[0]))

In [52]:
import numpy
def df2latex(df):
    
    print("\\begin{table}[H]")
    print("\\caption{CAPTION}")
    
    s = "\\begin{tabular}{"
    cols = df.columns
    for c in cols:
        s += "r "
    s=s.strip() + "}"
    print(s)
    print("\\toprule")
    header = " & ".join([ "\\textbf{" + c.replace("_"," ") + "} " for c in cols ])
    print(header + "\\\\")
    print("\\midrule")
    
    def val2str(val):
        if type(val) == float or type(val) == numpy.float64:
            return "{val:.3f}".format(val=val)
        else:
            print(type(val))
            return str(val)
    
    for i,row in df.iterrows():
        s = " & ".join([val2str(row[c]).replace(".000","") for c in cols])
        print(s + "\\\\")
            
    print("\\bottomrule")
    print("\\end{tabular}")
    print("\\label{table:LABEL}")
    print("\\end{table}")


## RNN

### CB

#### Hyper Param Tuning

In [53]:
params = "num_rnns,hidden_size"
df = get_df_sorted_by_f1score("CR_CB_TAGGING_VD_MOST_COMMON_TAG_RNN", params)
del df["micro_recall"]
del df["micro_precision"]
df2latex(df)

\begin{table}[H]
\caption{CAPTION}
\begin{tabular}{r r r}
\toprule
\textbf{micro f1 score}  & \textbf{num rnns}  & \textbf{hidden size} \\
\midrule
0.680 & 2 & 256\\
0.673 & 2 & 128\\
0.650 & 1 & 256\\
0.636 & 1 & 128\\
0.635 & 1 & 64\\
0.634 & 2 & 64\\
\bottomrule
\end{tabular}
\label{table:LABEL}
\end{table}


#### By Partition

In [54]:
colls = get_variants("CR_CB_TAGGING_TD_MOST_COMMON_TAG_RNN")
print_table(colls)


\begin{table}[H]
\caption{Bidirectional RNN Causal Relation Classification Accuracy on the Coral Bleaching Dataset}
\center{
\begin{tabular}{l r r r}
\toprule
\textbf{Data Partition} & \textbf{Micro-$F_1$} & \textbf{Micro-Recall} & \textbf{Micro-Precision}\\
\midrule

Training Data (CV)        &	0.869			&	0.871			&	0.866 \\
Validation Data (CV)      &	0.680			&	0.695			&	0.665 \\
Test Data                 &	0.676			&	0.656			&	0.698 \\

\bottomrule
\end{tabular}}
\label{table:rq2_bidirectional_rnn_cb}
\end{table}




### SC

#### Hyper Parameter Tuning

In [55]:
params = "num_rnns,hidden_size"
df = get_df_sorted_by_f1score("CR_SC_TAGGING_VD_MOST_COMMON_TAG_RNN", params)
del df["micro_recall"]
del df["micro_precision"]
df2latex(df)

\begin{table}[H]
\caption{CAPTION}
\begin{tabular}{r r r}
\toprule
\textbf{micro f1 score}  & \textbf{num rnns}  & \textbf{hidden size} \\
\midrule
0.769 & 2 & 256\\
0.756 & 2 & 128\\
0.754 & 1 & 256\\
0.752 & 2 & 64\\
0.742 & 1 & 128\\
0.730 & 1 & 64\\
\bottomrule
\end{tabular}
\label{table:LABEL}
\end{table}


In [56]:
colls = get_variants("CR_SC_TAGGING_TD_MOST_COMMON_TAG_RNN")
print_table(colls)


\begin{table}[H]
\caption{Bidirectional RNN Causal Relation Classification Accuracy on the Skin Cancer Dataset}
\center{
\begin{tabular}{l r r r}
\toprule
\textbf{Data Partition} & \textbf{Micro-$F_1$} & \textbf{Micro-Recall} & \textbf{Micro-Precision}\\
\midrule

Training Data (CV)        &	0.872			&	0.855			&	0.889 \\
Validation Data (CV)      &	0.769			&	0.761			&	0.777 \\
Test Data                 &	0.792			&	0.798			&	0.786 \\

\bottomrule
\end{tabular}}
\label{table:rq2_bidirectional_rnn_sc}
\end{table}




## Stacked Model

### CB

In [57]:
colls = get_variants("CR_CB_STACKED_VD")
print_table(colls)
colls


\begin{table}[H]
\caption{Stacked Model Causal Relation Classification Accuracy on the Coral Bleaching Dataset}
\center{
\begin{tabular}{l r r r}
\toprule
\textbf{Data Partition} & \textbf{Micro-$F_1$} & \textbf{Micro-Recall} & \textbf{Micro-Precision}\\
\midrule

Training Data (CV)        &	0.741			&	0.711			&	0.774 \\
Validation Data (CV)      &	0.695			&	0.656			&	0.738 \\
Test Data                 &	0.704			&	0.674			&	0.736 \\

\bottomrule
\end{tabular}}
\label{table:rq2_stacked_model_cb}
\end{table}




['CR_CB_STACKED_TD', 'CR_CB_STACKED_VD', 'TEST_CR_CB_STACKED_VD']

### SC

In [58]:
colls = get_variants("CR_SC_STACKED_VD")
print_table(colls)
colls


\begin{table}[H]
\caption{Stacked Model Causal Relation Classification Accuracy on the Skin Cancer Dataset}
\center{
\begin{tabular}{l r r r}
\toprule
\textbf{Data Partition} & \textbf{Micro-$F_1$} & \textbf{Micro-Recall} & \textbf{Micro-Precision}\\
\midrule

Training Data (CV)        &	0.801			&	0.757			&	0.850 \\
Validation Data (CV)      &	0.763			&	0.711			&	0.823 \\
Test Data                 &	0.765			&	0.719			&	0.816 \\

\bottomrule
\end{tabular}}
\label{table:rq2_stacked_model_sc}
\end{table}




['CR_SC_STACKED_TD', 'CR_SC_STACKED_VD', 'TEST_CR_SC_STACKED_VD']

## Parser Model

### CB

In [59]:
colls = [
    "CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_TD",
    "CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD",
    "TEST_CR_CB_SHIFT_REDUCE_PARSER_TEMPLATED_VD"
]
print_table(colls)


\begin{table}[H]
\caption{Shift Reduce Parser Causal Relation Classification Accuracy on the Coral Bleaching Dataset}
\center{
\begin{tabular}{l r r r}
\toprule
\textbf{Data Partition} & \textbf{Micro-$F_1$} & \textbf{Micro-Recall} & \textbf{Micro-Precision}\\
\midrule

Training Data (CV)        &	0.865			&	0.766			&	0.992 \\
Validation Data (CV)      &	0.720			&	0.671			&	0.778 \\
Test Data                 &	0.728			&	0.703			&	0.754 \\

\bottomrule
\end{tabular}}
\label{table:rq2_shift_reduce_parser_cb}
\end{table}




### SC

In [60]:
colls = [
    "CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_TD",
    "CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_HYPER_PARAM_VD",
    "TEST_CR_SC_SHIFT_REDUCE_PARSER_TEMPLATED_VD"
]
print_table(colls)


\begin{table}[H]
\caption{Shift Reduce Parser Causal Relation Classification Accuracy on the Skin Cancer Dataset}
\center{
\begin{tabular}{l r r r}
\toprule
\textbf{Data Partition} & \textbf{Micro-$F_1$} & \textbf{Micro-Recall} & \textbf{Micro-Precision}\\
\midrule

Training Data (CV)        &	0.879			&	0.790			&	0.992 \\
Validation Data (CV)      &	0.767			&	0.710			&	0.834 \\
Test Data                 &	0.790			&	0.760			&	0.823 \\

\bottomrule
\end{tabular}}
\label{table:rq2_shift_reduce_parser_sc}
\end{table}


